<a href="https://colab.research.google.com/github/Nithesh-b/miniproject2/blob/master/sentiment_analysis_naive_bayes_assignmentnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis using Naive Bayes
In this assignment, we will attempt to label tweets with sentiments (positive, neutral and negative) using Naive Bayes classifier, which has already been studied by you. Naive Bayes is a very basic approach to this problem, but gives surprisingly good accuracy sometimes. There are several elegant libraries for this problem, one of which will be briefly introduced in this notebook later. <br> 
<br> 
**Note:** Since Naive Bayes is a basic algorithm, a couple of very useful sklearn features have been incorporated in this assignment. They will help you write much more robust and clean code, and are applicable to any ML code you write. <br> 
References:
1. Pipeline: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
2. GridSearch: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

# Part 1 - Naive Bayes 

## 1. Importing required libraries - 2 Marks

In [151]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

# import pipeline, CountVectorizer, TfidfTransformer, GridSearchCV

## 2. Reading dataset 

In [152]:
data_t=pd.read_csv('/content/tweets.csv')
data_t.drop(data_t.columns[0],axis=1,inplace=True)
data_t.dropna
data_t.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


## 3. Text processing for the tweets [1+1= 2 Marks]

In [153]:
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')


stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing 
    
    # convert passed tweet to lower case 
    tweet = tweet.lower()

    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    
    # use work_tokenize imported above to tokenize the tweet
     
    
    
    return [word for word in tweet if word not in stopwords]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Process all tweets - 2 Marks

In [154]:
processed=[]

for tweet in data_t['tweets']:
    
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    
    #print(type(tweet))
    tweet = str(tweet)
    print(type(tweet))
    cleaned = processTweet(tweet)
    processed.append(' '.join(cleaned))
print (processed)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [155]:
data_t['processed']=processed
print(data_t)

                                                 tweets  ...                                          processed
0     Obama has called the GOP budget social Darwini...  ...  b   h   c l l e   h e   g p   b u g e   c l   ...
1     In his teen years, Obama has been known to use...  ...  n   h   e e n   e r   b   h   b e e n   k n w ...
2     IPA Congratulates President Barack Obama for L...  ...  p   c n g r u l e   p r e e n   b r c k   b   ...
3     RT @Professor_Why: #WhatsRomneyHiding - his co...  ...  r   A T U S E R   w h r n e h n g     h   c n ...
4     RT @wardollarshome: Obama has approved more ta...  ...  r   A T U S E R   b   h   p p r v e   r e   r ...
...                                                 ...  ...                                                ...
1375  @liberalminds Its trending idiot.. Did you loo...  ...  A T U S E R     r e n n g       u   l k     h ...
1376  RT @AstoldByBass: #KimKardashiansNextBoyfriend...  ...  r   A T U S E R   k k r h n n e x b f r e 

## 4. Create pipeline and define parameters for GridSearch

In [156]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

## 5. Split data into test and train [1 Mark]

In [157]:
nltk.download('senseval')

[nltk_data] Downloading package senseval to /root/nltk_data...
[nltk_data]   Package senseval is already up-to-date!


True

In [158]:
# split data into train and test with split as 0.2 
#from nltk.corpus import senseval
#instances = senseval.instances('hard.pos')
#size = int(len(instances) * 0.2)
#train_set, test_set = instances[size:], instances[:size]

In [159]:
print (type(data_t))

<class 'pandas.core.frame.DataFrame'>


In [160]:
x = data_t.tweets
y = data_t.processed

In [161]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

## 6. Perform classification (using GridSearch) - [3 Marks]

In [162]:
# perform GridSearch CV with 10 fold CV using pipeline and tuned_paramters defined above 

grid_search = GridSearchCV(text_clf, tuned_parameters, n_jobs=-1, verbose=1)
print("Performing grid search...")
print("pipeline:", [name for name, _ in text_clf.steps])
print("parameters:")
print(tuned_parameters)
#grid_search.fit(x_train, y_train)

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'vect__ngram_range': [(1, 1), (1, 2), (2, 2)], 'tfidf__use_idf': (True, False), 'tfidf__norm': ('l1', 'l2'), 'clf__alpha': [1, 0.1, 0.01]}


### Classification report - [2 Marks]

In [163]:
# print classification report after predicting on test set with best model obtained in GridSearch


## Important and interesting insight:

In [164]:
counts = data_t.labels.value_counts()
print(counts)

0    947
1    352
2     81
Name: labels, dtype: int64


We can see above that the class distribution is highly imbalanced, this would not lead to good sampling of the data for the classifier. For your learning, you could use SMOTE (https://imbalanced-learn.readthedocs.io/en/stable/api.html) to oversample the minority classes and then evaluate the performance with Naive Bayes and compare. 

# Part 2 - VADER sentiment analysis

**Valence Aware Dictionary and Sentiment Reasoner (VADER)** is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. VADER does not requires any training data but is constructed from a generalizable, valence-based, human-curated gold standard sentiment lexicon. (A sentiment lexicon is a list of lexical features e.g., words, which are generally labelled according to their semantic orientation as either positive or negative.). VADER has been found to be quite successful when dealing with social media texts, editorials, movie reviews, and product reviews. This is because VADER not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.

[Original Paper](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) <br> 
<br> 
( Install the library using `pip install vaderSentiment`)

In [165]:
!pip install vaderSentiment

In [166]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [167]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{} {}".format(sentence, str(score)))

Let's see how it performs on a custom sentence

In [168]:
sentiment_analyzer_scores("VADER is smart, handsome, and funny.")

VADER is smart, handsome, and funny. {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}


1. The Positive, Negative and Neutral scores represent the proportion of text that falls in these categories. This means our sentence was rated as 75% Positive, 25% Neutral and 0% Negative. Hence all these should add up to 1.

2. The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate. 

        positive sentiment: compound score >= 0.05
        neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
        negative sentiment: compound score <= -0.05

### 1. Punctuation

The use of an exclamation mark(!), increases the magnitude of the intensity without modifying the semantic orientation. For example, “The food here is good!” is more intense than “The food here is good.” and an increase in the number of (!), increases the magnitude accordingly.

In [169]:
# Baseline sentence
sentiment_analyzer_scores("The food here is good")

The food here is good {'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}


In [170]:
# Punctuation
print(sentiment_analyzer_scores("The food here is good!"))
print(sentiment_analyzer_scores("The food here is good!!"))
print(sentiment_analyzer_scores("The food here is good!!!"))

The food here is good! {'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.4926}
None
The food here is good!! {'neg': 0.0, 'neu': 0.534, 'pos': 0.466, 'compound': 0.5399}
None
The food here is good!!! {'neg': 0.0, 'neu': 0.514, 'pos': 0.486, 'compound': 0.5826}
None


### 2. Capitalization
Using upper case letters to emphasize a sentiment-relevant word in the presence of other non-capitalized words, increases the magnitude of the sentiment intensity. For example, “The food here is GREAT!” conveys more intensity than “The food here is great!”

In [171]:
# Baseline sentence
sentiment_analyzer_scores("The food here is great!")

The food here is great! {'neg': 0.0, 'neu': 0.477, 'pos': 0.523, 'compound': 0.6588}


In [172]:
# Capitalisation
sentiment_analyzer_scores("The food here is GREAT!")

The food here is GREAT! {'neg': 0.0, 'neu': 0.438, 'pos': 0.562, 'compound': 0.729}


### 3. Conjunctions
Use of conjunctions like “but”, signals a shift in sentiment polarity, with the sentiment of the text following the conjunction being dominant. “The food here is great, but the service is horrible” has mixed sentiment, with the latter half dictating the overall rating.

In [173]:
# Baseline sentence
sentiment_analyzer_scores("The food here is great")

The food here is great {'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'compound': 0.6249}


In [174]:
# Conjunctions
sentiment_analyzer_scores("The food here is great, but the service is horrible")

The food here is great, but the service is horrible {'neg': 0.31, 'neu': 0.523, 'pos': 0.167, 'compound': -0.4939}
